## Leaf Detector

In [1]:
# Importing necessary modules
import os
import tensorflow as tf
import tqdm

# Make sure, you have installed tensorpack library using pip instlal tensorpack
from tensorpack.predict import OfflinePredictor, PredictConfig
from tensorpack.tfutils import SmartInit, get_tf_version_tuple

# Importing local libraries
from dataset import register_coco, register_balloon
from config import config as cfg
from config import finalize_configs
from modeling.generalized_rcnn import ResNetC4Model, ResNetFPNModel
from functions_predict import *

In [2]:
# Download and load the weights file
load_weights = "model_weights/model-64000.data-00000-of-00001"

In [3]:
# Add COCO-formatted datasets to the registry
register_coco(cfg.DATA.BASEDIR)
register_balloon(cfg.DATA.BASEDIR)

In [4]:
# Loading the model
MODEL = ResNetFPNModel() if cfg.MODE_FPN else ResNetC4Model()

In [5]:
# Testing for availability of hardware
if not tf.test.is_gpu_available():
    from tensorflow.python.framework import test_util
    assert get_tf_version_tuple() >= (1, 7) and test_util.IsMklEnabled(), \
        "Inference requires either GPU support or MKL support!"

In [6]:
finalize_configs(is_training=False)

[0312 16:38:00 @config.py:319] Config: ------------------------------------------
{'BACKBONE': {'FREEZE_AFFINE': False,
              'FREEZE_AT': 2,
              'NORM': 'FreezeBN',
              'RESNET_NUM_BLOCKS': [3, 4, 23, 3],
              'STRIDE_1X1': False,
              'TF_PAD_MODE': False,
              'WEIGHTS': ''},
 'CASCADE': {'BBOX_REG_WEIGHTS': [[10.0, 10.0, 5.0, 5.0], [20.0, 20.0, 10.0, 10.0],
                                  [30.0, 30.0, 15.0, 15.0]],
             'IOUS': [0.5, 0.6, 0.7]},
 'DATA': {'ABSOLUTE_COORD': True,
          'BASEDIR': '',
          'CLASS_NAMES': ['BG', 'leaf'],
          'NUM_CATEGORY': 1,
          'NUM_WORKERS': 5,
          'TRAIN': ('coco_train2017',),
          'VAL': ('coco_val2017',)},
 'FPN': {'ANCHOR_STRIDES': (4, 8, 16, 32, 64),
         'CASCADE': False,
         'FRCNN_CONV_HEAD_DIM': 256,
         'FRCNN_FC_HEAD_DIM': 1024,
         'FRCNN_HEAD_FUNC': 'fastrcnn_2fc_head',
         'MRCNN_HEAD_FUNC': 'maskrcnn_up4conv_head'

In [7]:
# Changing certain config values for testing
cfg.TEST.RESULT_SCORE_THRESH = cfg.TEST.RESULT_SCORE_THRESH_VIS

In [8]:
# Create prediction config
predcfg = PredictConfig(
    model=MODEL,
    session_init=SmartInit(load_weights),
    input_names=MODEL.get_inference_tensor_names()[0],
    output_names=MODEL.get_inference_tensor_names()[1])

[0312 16:38:00 @varmanip.py:211] Checkpoint path model_weights/model-56000.data-00000-of-00001 is auto-corrected to model_weights/model-56000.
[0312 16:38:00 @sesscreate.py:38] WRN User-provided custom session config may not work due to TF bugs. See https://github.com/tensorpack/tensorpack/issues/497 for workarounds.


In [9]:
# Initializer Predictor
predictor = OfflinePredictor(predcfg)


[0312 16:38:00 @registry.py:90] 'conv0': [1, 3, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'pool0': [1, 64, ?, ?] --> [1, 64, ?, ?]

[0312 16:38:00 @registry.py:90] 'group0/block0/conv1': [1, 64, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block0/conv2': [1, 64, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block0/conv3': [1, 64, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block0/convshortcut': [1, 64, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block1/conv1': [1, 256, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block1/conv2': [1, 64, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block1/conv3': [1, 64, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block2/conv1': [1, 256, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block2/conv2': [1, 64, ?, ?] --> [1, 64, ?, ?]
[0312 16:38:00 @registry.py:90] 'group0/block2/conv3': [1,

[0312 16:38:02 @registry.py:90] 'group2/block18/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:02 @registry.py:90] 'group2/block18/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:02 @registry.py:90] 'group2/block18/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0312 16:38:02 @registry.py:90] 'group2/block19/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:02 @registry.py:90] 'group2/block19/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:02 @registry.py:90] 'group2/block19/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0312 16:38:03 @registry.py:90] 'group2/block20/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:03 @registry.py:90] 'group2/block20/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:03 @registry.py:90] 'group2/block20/conv3': [1, 256, ?, ?] --> [1, 1024, ?, ?]
[0312 16:38:03 @registry.py:90] 'group2/block21/conv1': [1, 1024, ?, ?] --> [1, 256, ?, ?]
[0312 16:38:03 @registry.py:90] 'group2/block21/conv2': [1, 256, ?, ?] --> [1, 256, ?, ?]
[03

In [10]:
# List out all test images
input_path = "test_images"
output_path = 'test_inferences'
test_images = os.listdir(input_path)

In [11]:
# Predicting on each input image

for input_file in test_images: #input_file = test_images[0]
    do_predict(predictor, input_path, input_file, output_path)

[0312 16:38:12 @functions_predict.py:101] Inference output for FS547-4-big.jpg Successful
[0312 16:38:13 @functions_predict.py:101] Inference output for 1-54_g7i2.png Successful
[0312 16:38:14 @functions_predict.py:101] Inference output for 1-7_g7i0.png Successful
[0312 16:38:14 @functions_predict.py:101] Inference output for 101614330.jpg Successful
[0312 16:38:15 @functions_predict.py:101] Inference output for 14472406_f1024.webp Successful
[0312 16:38:16 @functions_predict.py:101] Inference output for 15.jpg Successful
[0312 16:38:17 @functions_predict.py:101] Inference output for 16.jpg Successful
[0312 16:38:19 @functions_predict.py:101] Inference output for 19.jpg Successful
[0312 16:38:19 @functions_predict.py:101] Inference output for 22.jpg Successful
[0312 16:38:21 @functions_predict.py:101] Inference output for 27-500x375.jpg Successful
[0312 16:38:22 @functions_predict.py:101] Inference output for 30.jpg Successful
[0312 16:38:22 @functions_predict.py:101] Inference output 